In [19]:
import rqdatac as rq
from rqdatac import *
rq.init()

In [20]:
from functions_ti import *
from functions import *
import numpy as np
import pandas as pd

In [44]:
def get_acc_41(ics, start_date, end_date,iwl = 30, fh = 1, frequency = '1d', price_type = 'close'):
    a_train_rf, a_test_rf = [],[]
    a_train_log, a_test_log = [],[]
    a_train_ann, a_test_ann = [],[]
    a_train_lstm, a_test_lstm = [],[]
    a_train_svm, a_test_svm = [],[]

    ps = get_price(ics, start_date, end_date, frequency = frequency)[price_type]
    X_svm, y_svm = get_dataframe(ics, iwl, fh, start_date, end_date,frequency, classnum = 2, method = 'SVM')
    for i in np.r_[0:len(ps.index)-1000:250]:
        p = ps[i:i+1000]
        xs = X_svm[i:i+1000]
        print(type(p))
        rt = (p - p.shift(1))/p
        med = rt.median()
        r = np.concatenate((np.r_[1:21],np.r_[40:241:20]))
        result = []
        for m in r:
            rt = (p/p.shift(m)) - 1
            result.append(rt)
        x = list(np.array(result).T)
        if (len(x) != len(xs)):
            print("i = ", i)
            continue
        r = np.r_[1:241]
        result = []
        for m in r:
            rt = (p/p.shift(m)) - 1
            result.append(rt)
        x_lstm = list(np.array(result).T)
        x_lstm = np.concatenate((x_lstm, xs), axis = 1)
        x = np.concatenate((x, xs), axis = 1)
        X_train, X_test = [], []
        y_train, y_test = [], []
        X_train_lstm, X_test_lstm = [],[]

        y = (rt > med) * 1
        X_train += list(x)[241:749]
        y_train += list(y)[242:750]
        X_test += list(x)[750:998]
        y_test += list(y)[751:999]
        X_train_lstm += list(x_lstm)[241:749]
        X_test_lstm += list(x_lstm)[750:998]
        y_train_oh = keras.utils.to_categorical(y_train, num_classes=2)
        y_test_oh = keras.utils.to_categorical(y_test, num_classes=2)
        if (len(X_train) != len(y_train)):
            print('i:',i)
            continue
        model_svm = svm()
        model_lstm = lstm_250()
        model_ann = ann_41()
        model_log = LogisticRegression()
        model_rf = rf()

        model_log.fit(X_train, y_train)
        model_rf.fit(X_train, y_train)
        model_svm.fit(X_train, y_train)
        model_lstm.fit(sandlize(np.array(X_train_lstm)), y_train_oh,epochs = 50)
        model_ann.fit(np.array(X_train), y_train_oh,epochs = 200)
        a_train_rf.append(accuracy_score(y_train, model_rf.predict(X_train)))
        a_test_rf.append(accuracy_score(y_test, model_rf.predict(X_test)))
        a_train_log.append(accuracy_score(y_train, model_log.predict(X_train)))
        a_test_log.append(accuracy_score(y_test, model_log.predict(X_test)))
        a_train_ann.append(model_ann.evaluate(np.array(X_train), y_train_oh)[1])
        a_test_ann.append(model_ann.evaluate(np.array(X_test), y_test_oh)[1])
        a_train_lstm.append(model_lstm.evaluate(sandlize(np.array(X_train_lstm)), y_train_oh)[1])
        a_test_lstm.append(model_lstm.evaluate(sandlize(np.array(X_test_lstm)), y_test_oh)[1])
        a_train_svm.append(accuracy_score(y_train, model_svm.predict(X_train)))
        a_test_svm.append(accuracy_score(y_test, model_svm.predict(X_test)))
    df3 = pd.DataFrame(index = ['RAF','LOG','ANN','LSTM','SVM'])
    df3['Training accuracy'] = [np.average(a_train_rf),np.average(a_train_log),np.average(a_train_ann), np.average(a_train_lstm),np.average(a_train_svm)]
    df3["Testing accuracy"] = [np.average(a_test_rf), np.average(a_test_log), np.average(a_test_ann), np.average(a_test_lstm),np.average(a_train_svm)]
    return df3

def ann_41():
#     return an artificial neural network model, the construction is 31, 31, 10, 5, 2, activate function are ReLU except the out layer(softmax)
    model = Sequential()
    model.add(Dense(31, activation = 'relu', input_shape = (41, )))
    model.add(Dropout(0.1))
    model.add(Dense(10, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(5, activation = 'relu'))
    model.add(Dense(2, activation = 'softmax', bias_regularizer=regularizers.l1(0.00001)))
    adadelta = Adadelta(lr=1.0, rho=0.95, epsilon=1e-06)
    model.compile(loss='binary_crossentropy',
              optimizer = adadelta, metrics=['binary_accuracy'])
    return model

def panel_append(p):
#   define a function for each element from a panel to append other elements and form a list 
    result = []
    r = np.concatenate((np.r_[1:21],np.r_[40:241:20]))
    for m in r:
        rt = (p/p.shift(m)) - 1
        result.append(rt)
    return list(np.array(result).T)

def lstm_250():
#     return a lstm model
    model = Sequential()
    model.add(LSTM(25, return_sequences = False,
                   input_shape=(250, 1)))  # returns a sequence of vectors of dimension 32
    model.add(Dropout(0.1))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['binary_accuracy'])
    return model
def svm():
    gammas, Cs = [], []
    for a in np.r_[-15: 4][::2]:
        gamma = pow(float(2), a)
        gammas.append(gamma)
    for b in np.r_[-5: 16][::2]:
        c = pow(float(2), b)
        Cs.append(c)
    # Set the parameters by cross-validation  
    tuned_parameters = {'kernel': ['rbf'], 'gamma': gammas,  'C': Cs}
    clf = GridSearchCV(SVC(), tuned_parameters, cv = 5,
                  scoring = 'accuracy')
    return clf

In [45]:
classnum = 2
ics = '000001.XSHG'
start_date = '20060101'
end_date ='20180709'
iwl = 30
fh = 1
freq = '1d'
split = 0.8

In [46]:
print(1)

1


In [ ]:
df_dichan = get_acc_41('000006.XSHG', start_date, end_date,iwl = 30, fh = 1, frequency = '1d', price_type = 'close')

<class 'pandas.core.series.Series'>
Epoch 1/50
508/508 [==============================] - 6s - loss: 0.6572 - binary_accuracy: 0.6398     
Epoch 2/50
508/508 [==============================] - 4s - loss: 0.5780 - binary_accuracy: 0.7539     
Epoch 3/50
508/508 [==============================] - 4s - loss: 0.5167 - binary_accuracy: 0.7953     
Epoch 4/50
508/508 [==============================] - 4s - loss: 0.4323 - binary_accuracy: 0.8465     
Epoch 5/50
508/508 [==============================] - 3s - loss: 0.3153 - binary_accuracy: 0.8858     
Epoch 6/50
508/508 [==============================] - 4s - loss: 0.2637 - binary_accuracy: 0.9370     
Epoch 7/50
508/508 [==============================] - 4s - loss: 0.2067 - binary_accuracy: 0.9823     
Epoch 8/50
508/508 [==============================] - 4s - loss: 0.1605 - binary_accuracy: 0.9921     
Epoch 9/50
508/508 [==============================] - 3s - loss: 0.1344 - binary_accuracy: 0.9902     
Epoch 10/50
508/508 [================

In [ ]:
df_dichan